In [1]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Function
import torchvision
import time
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as util_data
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models
import cv2
import time

import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from typing import Any
import random
import torchvision

In [2]:
dataset_root = '/kaggle/input/palm-random/plam_6000'
#dataset_root = ‘../input/datagood’

dataset_train_root = os.path.join(dataset_root, 'train')
dataset_val_root = os.path.join(dataset_root, 'test/')
dataset_all_dataset_root = os.path.join(dataset_root, 'all_dataset')


# win设为0 
num_workers = 8

num_threads =8 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
num_epochs =100
# epoch_lr_decrease = 100
learning_rate = 0.004
encode_length =256
num_classes =600

In [4]:
# 配置训练集DataSet类
class SessionDataset(Dataset):
    
    # root数据集根目录
    def __init__(self, root, transform=None):
        # 对继承自父类的属性进行初始化
#         super(Oxford102Dataset, self).__init__()
        
        #读取图像标签  [  0   0   0 ... 599 599 599]
        self.labels = np.array([int(x.split('_')[0]) for x in os.listdir(path=root)])

        # 构造图像路径

        self.image_files = np.array([x.path for x in os.scandir(path=root)])
        
        # 数据增强
        self.transform = transform
        
    
    def __getitem__(self, index):
        
        # 根据Index读取图像
        img = cv2.imread(self.image_files[index])
        
        #根据index读取标签
        label = self.labels[index]


        if self.transform:
            img = self.transform(img)
        
        return img, label
    
    
    def __len__(self):
        return np.size(self.image_files)
        

In [5]:
data_transform = {
    "train": transforms.Compose([
        transforms.ToPILImage(),#将数据类型转变为图片数据
#         transforms.RandomResizedCrop((224, 224)),#随机裁剪224*224
        transforms.RandomResizedCrop((128, 128)),#随机裁剪224*224
        transforms.RandomHorizontalFlip(),#水平防线随机反转
        transforms.ToTensor(),#转为Tensor
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626)),#标准化处理
        
    ]),
    "val": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    "all_dataset": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    
}

In [6]:
train_batch_size = 64#64
val_batch_size = 64




session_train_dataset = SessionDataset(root=dataset_train_root,
                                    transform=data_transform['train'])

train_loader = DataLoader(session_train_dataset,
                         batch_size=train_batch_size,
                         shuffle=True,
                         num_workers=num_workers)


session_val_dataset = SessionDataset(root=dataset_val_root,
                                    transform=data_transform['val'])
val_loader = DataLoader(session_val_dataset,
                         batch_size=val_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

# 包含整个验证集

sessiondatabaseDataset = SessionDataset(root=dataset_all_dataset_root,
                                    transform=data_transform['all_dataset'])

database_loader = DataLoader(sessiondatabaseDataset,
                         batch_size=train_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

train_num = len(session_train_dataset)#4800
val_num = len(session_val_dataset)#1200
datasetloader_num = len(sessiondatabaseDataset)#6000
print(train_num, val_num,datasetloader_num)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


4800 1200 6000


In [7]:
import numpy as np

 


def logistic(x, k=1, x0=0):
    """
    Logistic function: L / (1 + exp(-k * (x - x0)))
    
    Parameters:
    - x: Input values
    - k: Steepness of the curve (default: 1)
    - x0: x-value of the sigmoid's midpoint (default: 0)
    
    Returns:
    - Result of the logistic function
    """
    return 1 / (1 + np.exp(-k * (x - x0)))


def new_dropout(x, level=0.5):  
    if level < 0. or level >= 1:#level是概率值，必须在0~1之间  
        raise Exception('Dropout level must be in interval [0, 1].')  
    retain_prob = 1. - level  
    #我们通过binomial函数，生成与x一样的维数向量。binomial函数就像抛硬币一样，我们可以把每个神经元当做抛硬币一样  
    #硬币 正面的概率为p，n表示每个神经元试验的次数  
    #因为我们每个神经元只需要抛一次就可以了所以n=1，size参数是我们有多少个硬币。 
     
   # sample=nn.random.binomial(n=1,p=retain_prob,size=x.shape)#即将生成一个0、1分布的向量，0表示这个神经元被屏蔽，不工作了，也就是dropout了  
    sample = torch.from_numpy(np.array(logistic(0.3, 1000))).to(device)

#
  #  sample=np.array(logistic(0.36,x.dim()))#
#     print (sample)  
#     print(type(sample))
    
    x *=sample          #屏蔽某些神经元
    
    x /= retain_prob   #此处是dropout
    return x



In [8]:
import torch
from torch import nn

__all__ = ['iresnet50_cbam']

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.channel_gate = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_gate = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel-wise attention
        channel_avg = self.channel_gate(x)
        x = x * channel_avg

        # Spatial-wise attention
        channel_max = self.spatial_gate(x)
        x = x * channel_max

        return x

class IBasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        self.bn1 = nn.BatchNorm2d(inplanes, eps=1e-05,)
        self.conv1 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out

class IResNet(nn.Module):
    fc_scale = 7 * 7

    def __init__(self, block, layers, dropout=0, num_features=512, zero_init_residual=False, groups=1, width_per_group=64, replace_stride_with_dilation=None, fp16=False):
        super(IResNet, self).__init__()
        self.fp16 = fp16
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=1e-05)
        self.prelu = nn.PReLU(self.inplanes)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.bn2 = nn.BatchNorm2d(512 * block.expansion, eps=1e-05,)
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        self.cbam = CBAM(512 * block.expansion)
        self.fc = nn.Linear(4 * 512 * block.expansion * self.fc_scale, num_features)
        self.features = nn.BatchNorm1d(num_features, eps=1e-05)
        nn.init.constant_(self.features.weight, 1.0)
        self.features.weight.requires_grad = False

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0, 0.1)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, IBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion, eps=1e-05, ),
            )
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        with torch.cuda.amp.autocast(self.fp16):
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.prelu(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.bn2(x)
            x = self.dropout(x)
            x = self.cbam(x)
            x = torch.flatten(x, 1)
        x = self.fc(x.float() if self.fp16 else x)
        x = self.features(x)
        return x

def iresnet50_cbam(pretrained=False, progress=True, **kwargs):
    return IResNet(IBasicBlock, [3, 4, 14, 3], **kwargs)

 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the basic residual block
class BasicBlock(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = F.relu(out)
        out = self.bn2(self.conv2(out))
        out = F.relu(out)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# Define the ResNet-50 model
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        layers = []
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# Create an instance of the ResNet-50 model
def original_resnet50(pretrained=False, progress=True, **kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)

In [10]:


# new layer
class hash(Function):
    @staticmethod
    def forward(ctx, input):
        #ctx.save_for_backward(input)
        #使用在pytroch中的sign函数得到二值哈希码
        return torch.sign(input)    
    @staticmethod
    def backward(ctx, grad_output):
        #input,  = ctx.saved_tensors
        #grad_output = grad_output.data
        return grad_output
def hash_layer(input):
    return hash.apply(input)


    
    
class CNN_ResNet(nn.Module):
    def __init__(self, encode_length, num_classes):
        super(CNN_ResNet, self).__init__()
        self.res = iresnet50_cbam(pretrained=False,num_features=1000).to(device)
    
        self.res.fc = nn.Linear(in_features=2048, out_features=1000, bias=True)
      
        self.fc_plus = nn.Linear(1000, encode_length) 
        
        self.fc = nn.Linear(encode_length, num_classes, bias=False)

    def forward(self, x):
        
        x = self.res(x)
        #x = new_dropout(x)
        x = self.fc_plus(x)

        code = hash_layer(x)
   
        output = self.fc(code)
        
  
        return output, x, code
    

#net = CNN_ResNet(encode_length=encode_length, num_classes=num_classes).to(device)
print('ok')

ok


In [11]:
import torch
import torch.nn as nn

class SecureRandomProjectionLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_classes):
        super(SecureRandomProjectionLayer, self).__init__()
        self.random_projection_matrices = nn.Parameter(torch.randn(num_classes, input_dim, output_dim) * 0.01, requires_grad=False)

    def forward(self, x, class_labels):
        class_specific_matrices = self.random_projection_matrices[class_labels]
        transformed_data = torch.matmul(x, class_specific_matrices)
        return transformed_data

class hash(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

def hash_layer(input):
    return hash.apply(input)

class CNN_ResNet(nn.Module):
    def __init__(self, encode_length, num_classes):
        super(CNN_ResNet, self).__init__()
        self.res = original_resnet50(pretrained=False, num_classes=1000).to(device)
        self.res.fc = nn.Linear(in_features=2048, out_features=1000)
        self.secure_projection = SecureRandomProjectionLayer(1000, 1000,num_classes)
        self.fc_plus = nn.Linear(1000, encode_length)
        self.fc = nn.Linear(encode_length, num_classes, bias=False)

    def forward(self, x, class_labels=None):
        x = self.res(x)
        if class_labels is not None:
            class_specific_matrices = self.secure_projection.random_projection_matrices[class_labels]
            x = torch.matmul(x, class_specific_matrices)
        if x.dim() != 3:
            x = x.view(x.size(0), -1, 1)

        n, j = x.shape[1], x.shape[2]
        x = x.view(x.size(0), n, j)
        max_values, _ = x.max(dim=2)
        x = self.fc_plus(max_values)
        code = hash_layer(x)
        
        # If class_labels are provided, use the class-specific projection matrix
        if class_labels is not None:
            class_specific_matrices = self.secure_projection.random_projection_matrices[class_labels]
            x = torch.matmul(x, class_specific_matrices)
        
        output = self.fc(code)
        
        return output, x, code

# Example usage
 
num_projection_classes =600
net = CNN_ResNet(encode_length=encode_length, num_classes=num_classes).to(device)
print('ok')

ok


In [12]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)




In [13]:
def compress(train, test, model, classes=600):
    retrievalB = list([])
    retrievalL = list([])
    with torch.no_grad():
        for batch_step, (data, target) in enumerate(train):         
            var_data = data.to(device)            
            _,_, code = model(var_data)        
            retrievalB.extend(code.cpu().data.numpy())
            retrievalL.extend(target)
         
        queryB=list([])
        queryL=list([])
        for batch_step, (data, target) in enumerate(test):     
            var_data = data.to(device)
            _,_, code = model(var_data)
            
            queryB.extend(code.cpu().data.numpy())
            queryL.extend(target)
            
        retrievalB=np.array(retrievalB)
        retrievalL=np.eye(classes)[np.array(retrievalL)]
            
        queryB=np.array(queryB)
        queryL=np.eye(classes)[np.array(queryL)]   
        
            
        return retrievalB, retrievalL, queryB, queryL


    
    
def calculate_hamming(B1, B2):
    """
    :param B1:  vector [n]
    :param B2:  vector [r*n]
    :return: hamming distance [r]
    """
    q = B2.shape[1] # max inner product value
    distH = 0.5 * (q - np.dot(B1, B2.transpose()))
    return distH


def calculate_map(qB, rB, queryL, retrievalL):
    """
       :param qB: {-1,+1}^{mxq} query bits
       :param rB: {-1,+1}^{nxq} retrieval bits
       :param queryL: {0,1}^{mxl} query label
       :param retrievalL: {0,1}^{nxl} retrieval label
       :return:
    """
    num_query = queryL.shape[0]
    map = 0
    for iter in range(num_query):
       
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        
        tsum = np.sum(gnd).astype(int)
        if tsum == 0:
            continue
        
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        count = np.linspace(1, tsum, tsum) # [1,2, tsum]
        tindex = np.asarray(np.where(gnd == 1)) + 1.0
        map_ = np.mean(count / (tindex))
       
        map = map + map_
    map = map / num_query
    return map



def calculate_top_map(qB, rB, queryL, retrievalL, topk):
    """
    :param qB: {-1,+1}^{mxq} query bits
    :param rB: {-1,+1}^{nxq} retrieval bits
    :param queryL: {0,1}^{mxl} query label
    :param retrievalL: {0,1}^{nxl} retrieval label
    :param topk:
    :return:
    """
    num_query = queryL.shape[0]
    topkmap = 0
    for iter in range(num_query):
        
       
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        tgnd = gnd[0:topk]
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        count = np.linspace(1, tsum, tsum)

        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        topkmap_ = np.mean(count / (tindex))
      
        topkmap = topkmap + topkmap_
    topkmap = topkmap / num_query
    return topkmap



def myCalcTopMap(rB, qB, retrievalL, queryL, topk):
    
    
    num_query = queryL.shape[0]
    
    topkmap = 0
    
    
    for iter in tqdm(range(num_query)):
        
        
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        
        
        hamm = CalcHammingDist(qB[iter, :], rB)
        
        
        ind = np.argsort(hamm)
        
       
        gnd = gnd[ind]
        
        
        tgnd = gnd[0:topk]
        
       
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        
       
        count = np.linspace(1, tsum, tsum)
        
        
        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        
        
        topkmap_ = np.mean(count / (tindex))
        
        topkmap = topkmap + topkmap_
   
    topkmap = topkmap / num_query
    
    return topkmap

In [ ]:
# 训练过程
# 训练过程
import matplotlib.pyplot as plt

# Initialize an empty list to store accuracy values for each epoch
epoch_accuracies = []
best = 0.0

torch.set_num_threads(num_threads)


# 根据条件判断是否使用多GPU
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)
    
net.to(device)


# Train the Model
acc_dict={}
for epoch in range(num_epochs):
    
    net.train()
#     adjust_learning_rate(optimizer, epoch)
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs, feature, _ = net(images)


        loss1 = criterion(outputs, labels)
        #loss2 = F.mse_loss(torch.abs(feature), Variable(torch.ones(feature.size()).cuda()))
#         loss2 = torch.mean(torch.abs(torch.pow(torch.abs(feature) - Variable(torch.ones(feature.size()).cuda()), 3)))
        loss2 = torch.mean(torch.abs(torch.pow(torch.abs(feature) - torch.ones(feature.size()).to(device), 3)))
        loss = loss1 + 0.1 * loss2
        loss.backward()
        optimizer.step()

#         if (i + 1) % (len(oxford102TrainDataset) // batch_size / 2) == 0:
        print ('Epoch [%d/%d], Iter [%d/%d] Loss1: %.4f Loss2: %.4f'
               % (epoch + 1, num_epochs, i + 1, len(sessiondatabaseDataset) // train_batch_size,
                  loss1.item(), loss2.item()))

    # Test the Model
    net.eval()  # Change model to 'eval' mode
    correct = 0
    total = 0
    with torch.no_grad():
        
        for images, labels in val_loader:
            images = images.to(device)
            # 取网络最后全连接层的参数
            outputs, _, _ = net(images)
            # 第一个参数为最大值，第二个是最大值的下标
            _, predicted = torch.max(outputs.cpu().data, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum()

    print('Test Accuracy of the model: %.2f %%' % (100.0 * correct / total))

    if 1.0 * correct / total > best:
        best = 1.0 * correct / total
        torch.save(net.state_dict(), 'temp44.pkl')
       
    
        
    print('best: %.2f %%' % (best * 100.0))



Epoch [1/100], Iter [1/93] Loss1: 6.4930 Loss2: 0.5262
Epoch [1/100], Iter [2/93] Loss1: 6.6363 Loss2: 0.5183
Epoch [1/100], Iter [3/93] Loss1: 6.5346 Loss2: 0.5049
Epoch [1/100], Iter [4/93] Loss1: 6.6652 Loss2: 0.5193
Epoch [1/100], Iter [5/93] Loss1: 6.6271 Loss2: 0.6041
Epoch [1/100], Iter [6/93] Loss1: 6.6572 Loss2: 0.6062
Epoch [1/100], Iter [7/93] Loss1: 6.5496 Loss2: 0.5426
Epoch [1/100], Iter [8/93] Loss1: 6.4623 Loss2: 0.5868
Epoch [1/100], Iter [9/93] Loss1: 6.6403 Loss2: 0.6169
Epoch [1/100], Iter [10/93] Loss1: 6.5559 Loss2: 0.6766
Epoch [1/100], Iter [11/93] Loss1: 6.6058 Loss2: 0.6019
Epoch [1/100], Iter [12/93] Loss1: 6.5780 Loss2: 0.6480
Epoch [1/100], Iter [13/93] Loss1: 6.5799 Loss2: 0.5600
Epoch [1/100], Iter [14/93] Loss1: 6.6209 Loss2: 0.5794
Epoch [1/100], Iter [15/93] Loss1: 6.7060 Loss2: 0.5863
Epoch [1/100], Iter [16/93] Loss1: 6.6230 Loss2: 0.6538
Epoch [1/100], Iter [17/93] Loss1: 6.6598 Loss2: 0.5742
Epoch [1/100], Iter [18/93] Loss1: 6.5859 Loss2: 0.7214
E

### 